# Problem Statement

In the realm of Neverland, where enchantment stretched far and wide, there lived a kind-hearted cattleman named Grassland Gus. His farm, Moo Meadows, was the sole source of the most delicious meat and milk in the kingdom. The people of Neverland would come from far and wide to procure these treasures.

Some procured these items directly from the farm at wholesale rates, while others obtained them from nearby groceries at retail prices. Fresh items were available at a premium, while frozen ones were sold at standard prices.

Gus packaged his dairy and meat in Enchanted Boxes. Each box held a different combination of meat and milk, and depending on their quality, some boxes were more valuable than others.

To purchase these magical boxes, the denizens of Neverland used Wishing Coins, which are tokens earned through acts of kindness. Every buyer had their own unique Magic Key, which kept track of all their purchases.

All exchanges of the kingdom are logged in the Enchanted Scroll, details of which are given in the file purchase.csv. The file contains records of purchases made over the last five months, including the date of purchase, the customer's magic key, the box ID purchased and purchase unit. Denizens select boxes to purchase from a list written on parchment. The dataset Boxes.csv enumerates all available boxes, including the box ID, product quality, delivery option, quantity of milk (cauldron), quantity of meat (stones) and box unit price.

There is no specific train.csv for this contest. Only **"purchase.csv"** and **"boxes.csv"** are given. You have to do everything from these two files.

**"problem 3.csv"** is given for you to predict, **"sample submission 3.csv"** is also there to help you about the submission template.

**“problem 3.csv”** contains the Magic Keys of customers who purchased at least one box of milk and/or meat in the first 15 days of March-2019. You need to predict what quantity of meat were be purchased by them in this period. Prepare and submit as submission.csv following the the template (sample submission 3.csv).

**Evaluation**
The evaluation metric for this problem is Root Mean Squared Error(RMSE). RMSE is calculated as the square root of the average of the squared differences between predicted and actual values

# Generic testing
This code is to test out the generic theory or eda for the problem datasets

In [11]:
import pandas as pd
import numpy as np
purchase = pd.read_csv("Problem 3/purchase.csv")
box = pd.read_csv("Problem 3/boxes.csv")


## Dropping ducplicates, nan values and impossible values

In [2]:
purchase = purchase.dropna().drop_duplicates() # Drop NaN values and duplicates
positive_box_count_mask = purchase['BOX_COUNT'] >= 0
purchase = purchase[positive_box_count_mask]
purchase['PURCHASE_DATE'] = pd.to_datetime(purchase['PURCHASE_DATE'], format='%d/%m/%Y')
purchase = purchase.sort_values(by='PURCHASE_DATE') # Sort purchase data by purchase date in ascending order
purchase.count()


PURCHASE_DATE    2455723
MAGIC_KEY        2455723
BOX_ID           2455723
BOX_COUNT        2455723
dtype: int64

# 0. New dynamic method:

## calculate_avg_time_between_purchases and feature_extraction

In [3]:
def calculate_avg_time_between_purchases(group):
    if len(group) > 1:
        return np.mean(group['PURCHASE_DATE'].diff().dt.days)
    else:
        return 150
    
def feature_extraction(purchase,grouped_df,box):
    # Task 1: Calculate the frequency of purchases for each Magic Key within specific time intervals (bi-weekly and monthly)
    print('1/6 Extracting Bi-Weekly and Monthly Purchase Count...')
    biweekly_purchase_count = purchase.groupby(['MAGIC_KEY', pd.Grouper(key='PURCHASE_DATE', freq='2W')]).size().unstack(fill_value=0)
    monthly_purchase_count = purchase.groupby(['MAGIC_KEY', pd.Grouper(key='PURCHASE_DATE', freq='ME')]).size().unstack(fill_value=0)

    # Task 2: Calculate the average time between purchases for each Magic Key
    print('2/6 Extracting Average Time between purchase...')
    # avg_time_between_purchases = grouped_df.apply(calculate_avg_time_between_purchases) 
    avg_time_between_purchases = 0

    # Task 3 days_since_last_purchase
    print('3/6 Extracting Days since last purchase...')
    last_purchase_date = grouped_df['PURCHASE_DATE'].max()
    days_since_last_purchase = (purchase['PURCHASE_DATE'].max() - last_purchase_date).dt.days.copy()
    
    # Task 4 purchase_count and total_spent
    print('4/6 Extracting purchase_count and total_spent...')
    merged_df = pd.merge(purchase, box, on='BOX_ID') 
    merged_df['SPENT'] = merged_df['BOX_COUNT'] * merged_df['UNIT_PRICE']
    grouped_df = merged_df.groupby('MAGIC_KEY') 
    purchase_count = grouped_df.size().rename('Purchase_Count') 
    total_spent = grouped_df['SPENT'].sum().rename('Total_Spent')
    # total_spent = grouped_df['UNIT_PRICE'].sum().rename('Total_Spent')

    # Task 5  total_milk_quantity & total_meat_quantity
    print('5/6 Extracting total_milk_quantity & total_meat_quantity...')
    total_milk_quantity = grouped_df['MILK'].sum().rename('Total_Milk_Quantity')
    total_meat_quantity = grouped_df['MEAT'].sum().rename('Total_Meat_Quantity')
    
    # Task 6 num_purchases_first_15_days and num_purchases_last_15_days
    # I will give it a try by dropping them too
    print('6/6 Extracting num_purchases_first_15_days and num_purchases_last_15_days...')
    first_15_days_purchase = merged_df[merged_df['PURCHASE_DATE'].dt.day <= 15]
    num_purchases_first_15_days = first_15_days_purchase.groupby(['MAGIC_KEY', first_15_days_purchase['PURCHASE_DATE'].dt.month]).size().groupby('MAGIC_KEY').sum()
    last_15_days_purchase = merged_df[merged_df['PURCHASE_DATE'].dt.day > 15]
    num_purchases_last_15_days = last_15_days_purchase.groupby(['MAGIC_KEY', last_15_days_purchase['PURCHASE_DATE'].dt.month]).size().groupby('MAGIC_KEY').sum()
    
    # Task 7 Most common box 
    print('$$ Extracting Most common box...')
    most_common_boxes = merged_df.groupby('MAGIC_KEY')['BOX_ID'].agg(lambda x: x.mode()[0])


    # Combine all features into a DataFrame
    features = pd.DataFrame({
        'Biweekly_Purchase_Count': biweekly_purchase_count.mean(axis=1),
        'Monthly_Purchase_Count': monthly_purchase_count.mean(axis=1),
        'Avg_Time_Between_Purchases': avg_time_between_purchases,
        'Days_Since_Last_Purchase': days_since_last_purchase
    })
    purchase_history_features = pd.concat([purchase_count, total_spent], axis=1) # Create a new DataFrame with purchase history features
    features = features.join(purchase_history_features, how='left')
    box_features_df = pd.concat([total_milk_quantity, total_meat_quantity], axis=1)
    features = features.join(box_features_df, how='left')
    features['Num_Purchases_First_15_Days'] = num_purchases_first_15_days     # Dropping 2 features 
    features['Num_Purchases_Last_15_Days'] = num_purchases_last_15_days       # Dropping 2 features
    features['Most_common_box'] = most_common_boxes
    features = features.fillna(0)
    features = features.drop(columns=['Avg_Time_Between_Purchases'])          # Dropping avg time between purchases 
    return features
    


## Set time boundary

In [4]:
purchase_oct_nov = purchase[(purchase['PURCHASE_DATE'].dt.year == 2018) &
                                    ((purchase['PURCHASE_DATE'].dt.month == 10) |
                                     (purchase['PURCHASE_DATE'].dt.month == 11))]
grouped_df_oct_nov = purchase_oct_nov.groupby('MAGIC_KEY') # Group by MAGIC_KEY
features = feature_extraction(purchase_oct_nov,grouped_df_oct_nov,box)


1/6 Extracting Bi-Weekly and Monthly Purchase Count...
2/6 Extracting Average Time between purchase...
3/6 Extracting Days since last purchase...
4/6 Extracting purchase_count and total_spent...
5/6 Extracting total_milk_quantity & total_meat_quantity...
6/6 Extracting num_purchases_first_15_days and num_purchases_last_15_days...
$$ Extracting Most common box...


## Labelling features

In [5]:
# Mean/median used labelling features  

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
def Labelling_features(features, purchase, box , end_date):

    start_date = pd.Timestamp('2018-10-01')
    end_date = pd.Timestamp(end_date)

    merged_df = pd.merge(purchase, box, on='BOX_ID')
    merged_df = merged_df[(merged_df['PURCHASE_DATE'] >= start_date) & (merged_df['PURCHASE_DATE'] <= end_date)]

    most_common_meat = merged_df.groupby('MAGIC_KEY')['MEAT'].median()
    most_common_meat_df = most_common_meat.to_frame(name='Most_Common_Box')

    features_with_labels = pd.merge(features, most_common_meat_df, left_index=True, right_index=True, how='left')
    features_with_labels = features_with_labels.dropna(subset=['Most_Common_Box'])
    features_with_labels = features_with_labels.rename(columns={'Most_Common_Box': 'labels'})

    return features_with_labels
    

In [6]:
# features = Labelling_features(features,purchase, 2018,12) # for december of 2018

features = Labelling_features(features,purchase, box, '2018-12-15') # for december of 2018

## Random Forest Regressor
This was used for final submission. Three step solution 
* Using data from OCT-NOV           ; Labels = DEC Half
* Using data from OCT-NOV-DEC       ; Labels = JAN Half
* Using data from OCT-NOV-DEC-JAN   ; Labels = FEB Half

In [8]:
# using RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
labels = features['labels'].to_numpy()
features_pure = (features.drop(columns=['labels'])).to_numpy()
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_pure)

X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.25, random_state=42)
classifier = RandomForestRegressor(verbose = 1)
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
 
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", rmse)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Root Mean Squared Error: 0.12039738479941026


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.5s finished


In [9]:
# Model training for oct_nov_dec
purchase_oct_nov_dec = purchase[(purchase['PURCHASE_DATE'].dt.year == 2018) &
                                    ((purchase['PURCHASE_DATE'].dt.month == 10) |
                                     (purchase['PURCHASE_DATE'].dt.month == 11) |
                                     (purchase['PURCHASE_DATE'].dt.month == 12))]
grouped_df_oct_nov_dec = purchase_oct_nov_dec.groupby('MAGIC_KEY') # Group by MAGIC_KEY
features = feature_extraction(purchase_oct_nov_dec,grouped_df_oct_nov_dec,box)

# Labelling 
print("Feature Labelling is going on...")
# features = Labelling_features(features,purchase, 2019, 1) # for  of 2019
features = Labelling_features(features,purchase, box, '2019-01-15') # for January of 2019

#defining elimentary things
labels = features['labels'].to_numpy()
features_pure = (features.drop(columns=['labels'])).to_numpy()
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_pure)

# Training
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.25, random_state=42)
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
 
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", rmse)

1/6 Extracting Bi-Weekly and Monthly Purchase Count...
2/6 Extracting Average Time between purchase...
3/6 Extracting Days since last purchase...
4/6 Extracting purchase_count and total_spent...
5/6 Extracting total_milk_quantity & total_meat_quantity...
6/6 Extracting num_purchases_first_15_days and num_purchases_last_15_days...
$$ Extracting Most common box...
Feature Labelling is going on...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Root Mean Squared Error: 0.1106643415604782


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.0s finished


In [10]:
# Model training for oct_nov_dec_jan
# Time boundary, OCT, Nov, Dec, Jan
purchase_oct_nov_dec_jan = purchase[((purchase['PURCHASE_DATE'].dt.year == 2018) | (purchase['PURCHASE_DATE'].dt.year == 2019)) &
                                    ((purchase['PURCHASE_DATE'].dt.month == 10) |
                                     (purchase['PURCHASE_DATE'].dt.month == 11) |
                                     (purchase['PURCHASE_DATE'].dt.month == 12) |
                                     (purchase['PURCHASE_DATE'].dt.month == 1))]
grouped_df_oct_nov_dec_jan = purchase_oct_nov_dec_jan.groupby('MAGIC_KEY') # Group by MAGIC_KEY
features = feature_extraction(purchase_oct_nov_dec_jan,grouped_df_oct_nov_dec_jan,box)

# Labelling 
# features = Labelling_features(features,purchase, 2019, 2) # for February of 2019
features = Labelling_features(features,purchase,box, '2019-02-15') # for February of 2019

#defining elimentary things
labels = features['labels'].to_numpy()
features_pure = (features.drop(columns=['labels'])).to_numpy()
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_pure)

# Training
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.25, random_state=42)
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
 
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", rmse)

1/6 Extracting Bi-Weekly and Monthly Purchase Count...
2/6 Extracting Average Time between purchase...
3/6 Extracting Days since last purchase...
4/6 Extracting purchase_count and total_spent...
5/6 Extracting total_milk_quantity & total_meat_quantity...
6/6 Extracting num_purchases_first_15_days and num_purchases_last_15_days...
$$ Extracting Most common box...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Root Mean Squared Error: 0.11113453094235855


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    7.0s finished


## Neural Network model defined 
Although this model wasn't used for final submission


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers

def define_model(n_input, n_hidden_layer, n_unit_per_layer,dropout_rate=0.25, l2_penalty=0.01):
    model = Sequential()
    model.add(Dense(n_unit_per_layer, input_shape=(n_input,), activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(l2_penalty)))
    model.add(Dropout(dropout_rate))

    # # Adding one additional hidden layer
    # model.add(Dense(n_unit_per_layer, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(l2_penalty)))
    # model.add(Dropout(dropout_rate))
    

    model.add(Dense(1, activation='linear'))  # 1 class, so output layer has 290 units and uses softmax activation
    return model

### Model train on OCT-NOV data. label DEC

In [ ]:
labels = features['labels'].to_numpy()
features_pure = (features.drop(columns=['labels'])).to_numpy()
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_pure)
num_node = 50

model_oct_nov = define_model(features_scaled.shape[1], 1, num_node)
model_oct_nov.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])  # Using Mean Squared Error as the loss function

X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.25, random_state=42)
hist_oct_nov = model_oct_nov.fit(X_train, y_train, epochs=50, batch_size=4096)
test_loss, test_accuracy = model_oct_nov.evaluate(X_test, y_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

model_oct_nov_weights = model_oct_nov.get_weights()


In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist_oct_nov.history['loss'], label='Training Loss')

plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

### Model train on OCT-NOV-DEC Label Jan

In [ ]:
# Time boundary, OCT, Nov, Dec
purchase_oct_nov_dec = purchase[(purchase['PURCHASE_DATE'].dt.year == 2018) &
                                    ((purchase['PURCHASE_DATE'].dt.month == 10) |
                                     (purchase['PURCHASE_DATE'].dt.month == 11) |
                                     (purchase['PURCHASE_DATE'].dt.month == 12))]
grouped_df_oct_nov_dec = purchase_oct_nov_dec.groupby('MAGIC_KEY') # Group by MAGIC_KEY
features = feature_extraction(purchase_oct_nov_dec,grouped_df_oct_nov_dec,box)

# Labelling 
print("Feature Labelling is going on...")
# features = Labelling_features(features,purchase, 2019, 1) # for  of 2019
features = Labelling_features(features,purchase, '2019-01-15') # for January of 2019

#defining elimentary things
labels = features['labels'].to_numpy()
features_pure = (features.drop(columns=['labels'])).to_numpy()
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_pure)

In [ ]:

# Model defining and setting previous weights
model_oct_nov_dec = define_model(features_scaled.shape[1], 1, num_node)
model_oct_nov_dec.set_weights(model_oct_nov_weights)
model_oct_nov_dec.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)
hist_oct_nov_dec = model_oct_nov_dec.fit(X_train, y_train, epochs=100, batch_size=4096)
test_loss, test_accuracy = model_oct_nov_dec.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

model_oct_nov_dec_weights = model_oct_nov_dec.get_weights()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist_oct_nov_dec.history['loss'], label='Training Loss')

plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

### Model train on OCT-NOV-DEC-Jan Label Feb

In [ ]:
# Time boundary, OCT, Nov, Dec, Jan
purchase_oct_nov_dec_jan = purchase[((purchase['PURCHASE_DATE'].dt.year == 2018) | (purchase['PURCHASE_DATE'].dt.year == 2019)) &
                                    ((purchase['PURCHASE_DATE'].dt.month == 10) |
                                     (purchase['PURCHASE_DATE'].dt.month == 11) |
                                     (purchase['PURCHASE_DATE'].dt.month == 12) |
                                     (purchase['PURCHASE_DATE'].dt.month == 1))]
grouped_df_oct_nov_dec_jan = purchase_oct_nov_dec_jan.groupby('MAGIC_KEY') # Group by MAGIC_KEY
features = feature_extraction(purchase_oct_nov_dec_jan,grouped_df_oct_nov_dec_jan,box)

# Labelling 
# features = Labelling_features(features,purchase, 2019, 2) # for February of 2019
features = Labelling_features(features,purchase, '2019-02-15') # for February of 2019

#defining elimentary things
labels = features['labels'].to_numpy()
features_pure = (features.drop(columns=['labels'])).to_numpy()
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_pure)

In [ ]:
# Model defining and setting previous weights
model_oct_nov_dec_jan = define_model(features_scaled.shape[1], 1, num_node)
model_oct_nov_dec_jan.set_weights(model_oct_nov_dec_weights)
model_oct_nov_dec_jan.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)
hist_oct_nov_dec_jan = model_oct_nov_dec_jan.fit(X_train, y_train, epochs=100, batch_size=4096)
test_loss, test_accuracy = model_oct_nov_dec_jan.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

model_oct_nov_dec_jan_weights = model_oct_nov_dec_jan.get_weights()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist_oct_nov_dec_jan.history['loss'], label='Training Loss')

plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

### Saving and loading weights

In [ ]:
# Assuming model is your TensorFlow model object

model_oct_nov_dec_jan.save_weights('model.weights.h5')

# model = define_model(features_scaled.shape[1], 1, num_node)
# model.load_weights('model_weights.h5')

# Check Feb test

In [15]:
# Filter data for the first 15 days of February 2019
start_date = pd.Timestamp('2019-02-01')
end_date = pd.Timestamp('2019-02-15')
merged_df = pd.merge(purchase, box, on='BOX_ID')
merged_df = merged_df[(merged_df['PURCHASE_DATE'] >= start_date) & (merged_df['PURCHASE_DATE'] <= end_date)]
selected_columns = ['MAGIC_KEY', 'MEAT']
feb_test = merged_df[selected_columns]
feb_test
# most_common_meat = merged_df.groupby('MAGIC_KEY')['MEAT'].median()
# most_common_meat_df = most_common_meat.to_frame(name='Most_Common_Box')

,MAGIC_KEY,MEAT
1867906,2918AD53FC5,2.4
1867907,28F9F1C4022,3.6
1867908,28F136BD796,3.6
1867909,2CEF3F9E019,2.9
1867910,2BE8E1C3686,1.8
...,...,...
2164942,2C7E8FEB228,2.0
2164943,2C08F604F7D,2.9
2164944,2CC80E6BA25,3.3
2164945,2BC9D82492E,0.0


In [16]:
grouped_df = purchase.groupby('MAGIC_KEY') # Group by MAGIC_KEY
features = feature_extraction(purchase,grouped_df,box)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
extracted_features_df = features.loc[feb_test['MAGIC_KEY']]
extracted_features_df

1/6 Extracting Bi-Weekly and Monthly Purchase Count...
2/6 Extracting Average Time between purchase...
3/6 Extracting Days since last purchase...
4/6 Extracting purchase_count and total_spent...
5/6 Extracting total_milk_quantity & total_meat_quantity...
6/6 Extracting num_purchases_first_15_days and num_purchases_last_15_days...
$$ Extracting Most common box...


,Biweekly_Purchase_Count,Monthly_Purchase_Count,Days_Since_Last_Purchase,Purchase_Count,Total_Spent,Total_Milk_Quantity,Total_Meat_Quantity,Num_Purchases_First_15_Days,Num_Purchases_Last_15_Days,Most_common_box
MAGIC_KEY,,,,,,,,,,
2918AD53FC5,0.166667,0.4,27,2,26.14,10.0,2.4,1.0,1.0,233.0
28F9F1C4022,0.166667,0.4,27,2,31.92,0.0,7.2,1.0,1.0,255.0
28F136BD796,0.250000,0.6,4,3,47.88,0.0,10.8,2.0,1.0,255.0
2CEF3F9E019,0.416667,1.0,6,5,107.90,84.0,11.2,4.0,1.0,143.0
2BE8E1C3686,0.250000,0.6,27,3,59.94,35.0,5.6,3.0,0.0,42.0
...,...,...,...,...,...,...,...,...,...,...
2C7E8FEB228,0.416667,1.0,13,5,99.90,80.0,9.4,1.0,4.0,143.0
2C08F604F7D,0.416667,1.0,13,5,103.90,68.0,10.1,1.0,4.0,137.0
2CC80E6BA25,0.416667,1.0,13,5,79.80,0.0,16.5,2.0,3.0,106.0


In [17]:
# suitable for neural networks
X_sub = scaler.transform(extracted_features_df)
predictions = classifier.predict(X_sub)
predictions

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.8s finished


array([1.41284149, 3.2655    , 3.126     , ..., 4.545     , 0.017     ,
       0.02      ])

In [18]:
rmse = np.sqrt(mean_squared_error(feb_test['MEAT'].values, predictions))
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 0.42746913436016526


# Submission

In [22]:
sample = pd.read_csv('Problem 3/sample submission 3.csv')
problem = pd.read_csv('Problem 3/problem 3.csv')

mean_values = features.mean()
new_row = pd.DataFrame([mean_values], index=['290D33249B7'], columns=features.columns)
features = pd.concat([features, new_row])

extracted_features_df = features.loc[problem['MAGIC_KEY']]
extracted_features_df


,Biweekly_Purchase_Count,Monthly_Purchase_Count,Days_Since_Last_Purchase,Purchase_Count,Total_Spent,Total_Milk_Quantity,Total_Meat_Quantity,Num_Purchases_First_15_Days,Num_Purchases_Last_15_Days,Most_common_box
2BCFE9C06A7,0.416667,1.0,0.0,5.0,240.00,50.0,18.0,0.0,5.0,56.0
2C2A872B5A2,0.166667,0.4,86.0,2.0,43.96,20.0,4.0,2.0,0.0,132.0
2C6A897671B,0.250000,0.6,0.0,3.0,71.94,31.0,6.6,0.0,3.0,162.0
2C6F1287F53,0.833333,2.0,0.0,10.0,431.94,102.0,31.8,2.0,8.0,56.0
2C658198CC9,0.250000,0.6,24.0,3.0,71.94,31.0,6.6,3.0,0.0,162.0
...,...,...,...,...,...,...,...,...,...,...
2C0804EFE49,0.083333,0.2,75.0,1.0,11.96,0.0,2.5,1.0,0.0,77.0
2C080B48630,0.083333,0.2,95.0,1.0,12.18,10.0,0.0,0.0,1.0,8.0
2C08243C58E,0.083333,0.2,28.0,1.0,11.96,0.0,2.4,0.0,1.0,3.0
2C082C78575,0.083333,0.2,126.0,1.0,12.98,10.0,1.8,0.0,1.0,17.0


In [24]:
# suitable for neural networks
X_sub = scaler.transform(extracted_features_df)
predictions = classifier.predict(X_sub)
predictions.shape

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


(5379,)

In [25]:
# Create a DataFrame with MAGIC_KEY and PURCHASE columns
submit = pd.DataFrame({'MAGIC_KEY': problem['MAGIC_KEY'], 'MEAT': predictions})
submit.to_csv('submit_p3_v3_.csv', index=False) 
submit.shape

(5379, 2)